# Simple Image Classifier with Pre-Trained image featurizer

To train traditional image classifiers, a large number of images is needed. By utilising the Pre-Trained image featurizer that comes with Microsoft ML Server, we can create a powerful image classifier with very limited training.

In [1]:
library(imager)

Loading required package: plyr
Loading required package: magrittr

Attaching package: 'imager'

The following object is masked from 'package:magrittr':

    add

The following object is masked from 'package:plyr':

    liply

The following objects are masked from 'package:stats':

    convolve, spectrum

The following object is masked from 'package:graphics':

    frame

The following object is masked from 'package:base':

    save.image



### Load Data

In [2]:
imageLocation = "Data/Pictures"
types <- list.files(file.path(imageLocation, "Train"))
print(types)
images<- c()
typeCount<-c()
for(type in types){
    subImage <-list.files(file.path(imageLocation, "Train", type))
    images <- c(images, file.path(imageLocation, "Train", type, subImage))
    typeCount <- c(typeCount, length(subImage))
}

#print(typeCount)
#print(rep(types, typeCount))

imagesDF <- data.frame(Image = images, stringsAsFactors = FALSE)
imagesDF$Label <- rep(types, typeCount)
print(head(imagesDF,10))

[1] "Cat"        "Dog"        "FighterJet" "Fish"       "Flower"    
[6] "Helicopter" "Monkey"     "Table"     
                               Image Label
1   Data/Pictures/Train/Cat/cat1.jpg   Cat
2   Data/Pictures/Train/Cat/cat2.jpg   Cat
3   Data/Pictures/Train/Cat/cat3.jpg   Cat
4   Data/Pictures/Train/Cat/cat4.jpg   Cat
5   Data/Pictures/Train/Cat/cat5.jpg   Cat
6   Data/Pictures/Train/Cat/cat6.JPG   Cat
7  Data/Pictures/Train/Dog/dog1.JPEG   Dog
8  Data/Pictures/Train/Dog/dog2.JPEG   Dog
9  Data/Pictures/Train/Dog/dog3.JPEG   Dog
10 Data/Pictures/Train/Dog/dog4.JPEG   Dog


### Train Model

In [3]:

imagesModel <- rxLogisticRegression(
  formula = Label~Features,
  data = imagesDF,
  type = "multiClass",
  mlTransforms = list(
    loadImage(vars = list(Features = "Image")),
    resizeImage(vars = "Features", width = 224, height = 224),
    extractPixels(vars = "Features"),
    featurizeImage(var = "Features", dnnModel = "resnet50")) 
)

Automatically adding a MinMax normalization transform, use 'norm=Warn' or 'norm=No' to turn this behavior off.
Beginning processing data.
Rows Read: 49, Read Time: 0.001, Transform Time: 0
Beginning processing data.
Automatically converting column 'Label' into a factor.
Beginning processing data.
Rows Read: 49, Read Time: 0.001, Transform Time: 0
Beginning processing data.
Beginning processing data.
Rows Read: 49, Read Time: 0, Transform Time: 0
Beginning processing data.
Beginning processing data.
Rows Read: 49, Read Time: 0, Transform Time: 0
Beginning processing data.
LBFGS multi-threading will attempt to load dataset into memory. In case of out-of-memory issues, turn off multi-threading by setting trainThreads to 1.
Beginning optimization
num vars: 16392
improvement criterion: Mean Improvement
L1 regularization selected 214 of 16392 weights.
Not training a calibrator because it is not needed.
Elapsed time: 00:01:35.7050285
Elapsed time: 00:00:01.0224914


### Define Predict function

In [4]:
classImage<-function(path){
  imgDF<-data.frame(Image = path, Label = "", stringsAsFactors = FALSE)
  prediction <- rxPredict(imagesModel, data = imgDF, extraVarsToWrite = list("Label", "Image"), reportProgress = 0)
  #typeOfImage <- imagesDF$Type[which(imagesDF$Label == prediction$PredictedLabel)[[1]]]
  typeOfImage <- prediction$PredictedLabel
  cat(paste("The image is of type: ", typeOfImage))
  return(typeOfImage)
}

In [5]:
path<- file.path(imageLocation, "test/fish.jpg")
classImage(path)

Elapsed time: 00:00:05.2749727
The image is of type:  Fish

[1] Fish
Levels: Cat Dog FighterJet Fish Flower Helicopter Monkey Table

### Publish model as web service

In [ ]:
remoteLogin("http://localhost:12800", 
            username = "admin", 
            password = "Pass@word1",
            session = FALSE)

api <- publishService(
     "classImage",
     code = classImage,
     model = imagesModel,
     inputs = list(Image = "character"),
     outputs = list(type = "character"),
     v = "v1.0.1"
)

# Pre-Trained Sentiment Analysis Model

In [6]:
 # Create the data. These phrases can be edited as you wish.
 CustomerReviews <- data.frame(Review = c(
   "I really did not like the taste of it",
   "It was surprisingly quite good!",
   "I will never ever ever go to that place again!!",
     "This phrase is neutral. Your own phrase here.",
     "I had a great time eating bland food and getting ripped off.",
   "Fantastic, Amazing, Wonderful, Perfect."),
   stringsAsFactors = FALSE)

 # Get the sentiment scores
 #sentimentScores <- rxFeaturize(data = CustomerReviews, 
 #                               mlTransforms = getSentiment(vars = list(SentimentScore = "Review")))

scoreSentiment <- function(yourString){
    stringDf <- data.frame(String = c(yourString), stringsAsFactors = FALSE)
    scoreOut <- rxFeaturize(data = stringDf, mlTransforms = getSentiment(vars = list(SentimentScore = "String")))
    score <- scoreOut$SentimentScore
    print(score)
    return(ifelse(score > 0.5, "Positive", "Negative"))
}

 # Let's translate the score to something more meaningful
 #sentimentScores$PredictedRating <- ifelse(sentimentScores$SentimentScore > 0.6, 
 #                                          "Positive", "Negative")

 # Let's look at the results
 #sentimentScores
for(i in CustomerReviews){
    print(scoreSentiment(i))
}

Beginning processing data.
Rows Read: 6, Read Time: 0, Transform Time: 0
Beginning processing data.
Elapsed time: 00:00:04.8370230
Finished writing 6 rows.
Writing completed.
[1] 0.4617899 0.9601924 0.3103435 0.5000000 0.3925641 0.9989264
[1] "Negative" "Positive" "Negative" "Negative" "Negative" "Positive"
